## 4.3 듀얼 모멘텀 전략
* 상대 모멘텀
    * 투자 자산 가운데 상대적으로 상승 추세가 강한 종목에 투자
* 절대 모멘텀
    * 과거 시점 대비 현재 시점의 절대적 상승세를 평가
* 듀얼 모멘텀
    * 게리 안토나치 창시
    * 상대 모멘텀과 절대 모멘텀 결합
### 4.3.1 듀얼 모멘텀 전략 구현을 위한 절대 모멘텀 전략

In [1]:
# pip install yfinance
import pandas as pd
import numpy as np
import FinanceDataReader as fdr
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime

In [2]:
# df = fdr.DataReader('US500', start='2000')
df = fdr.DataReader('spy', start='2000')
data = df.loc[:, ['Adj Close']].dropna().copy()
data.columns = ['Close']
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5911 entries, 2000-01-03 to 2023-06-30
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Close   5911 non-null   float64
dtypes: float64(1)
memory usage: 92.4 KB


In [3]:
# 말일 날짜 추출
data['STD_YM'] = data.apply(lambda x: x.name.strftime('%Y-%m'), axis=1)
month_last_df = data.drop_duplicates(['STD_YM'], keep="last").copy()
print(month_last_df.head())

                Close   STD_YM
Date                          
2000-01-31  90.773834  2000-01
2000-02-29  89.391716  2000-02
2000-03-31  98.055092  2000-03
2000-04-28  94.611313  2000-04
2000-05-31  93.123787  2000-05


In [4]:
# 1개월,12개월 전 값 추출
month_last_df['BF_1M_Close'] = month_last_df.shift(1)['Close']
month_last_df['BF_12M_Close'] = month_last_df.shift(12)['Close']
month_last_df.fillna(0, inplace=True)
month_last_df = month_last_df.loc['2008':]
print(month_last_df.head(15))

                 Close   STD_YM  BF_1M_Close  BF_12M_Close
Date                                                      
2008-01-31  101.562195  2008-01   108.097923    104.353470
2008-02-29   98.937584  2008-02   101.562195    102.306320
2008-03-31   98.052826  2008-03    98.937584    103.491989
2008-04-30  102.726288  2008-04    98.052826    108.076180
2008-05-30  104.279137  2008-05   102.726288    111.742096
2008-06-30   95.563919  2008-06   104.279137    110.108383
2008-07-31   94.705200  2008-07    95.563919    106.660904
2008-08-29   96.168724  2008-08    94.705200    108.029640
2008-09-30   87.112190  2008-09    96.168724    112.211922
2008-10-31   72.722443  2008-10    87.112190    113.734238
2008-11-28   67.660469  2008-11    72.722443    109.329056
2008-12-31   68.323296  2008-12    67.660469    108.097923
2009-01-30   62.712959  2009-01    68.323296    101.562195
2009-02-27   55.974522  2009-02    62.712959     98.937584
2009-03-31   60.637794  2009-03    55.974522     98.0528

In [5]:
# 포지션 기록
book = data['2008':].copy()
book['trade'] = ''
print(book.head())

                 Close   STD_YM trade
Date                                 
2008-01-02  107.151566  2008-01      
2008-01-03  107.099785  2008-01      
2008-01-04  104.475174  2008-01      
2008-01-07  104.386467  2008-01      
2008-01-08  102.700760  2008-01      


In [6]:
# 거래 실행, 매월 첫 영업일로 리밸런싱
ticker = 'S&P500'
for i in month_last_df.index:
    signal = ''
    momentum_index = month_last_df.loc[i,'BF_1M_Close'] / month_last_df.loc[i, 'BF_12M_Close'] - 1
    flag = True if ((momentum_index > 0.0) and (momentum_index != np.inf) and (momentum_index != -np.inf)) else False
    if flag:
        signal = 'buy ' + ticker
    print(f"날짜 : {i.strftime('%Y-%m-%d')}, 모멘텀 인덱스 : {momentum_index}, flag : {flag}, signal : {signal}")
    book.loc[i.strftime('%Y-%m'):, 'trade'] = signal # 매월 첫 영업일로 리밸런싱
print(book.loc[:'2008-02-01'])

날짜 : 2008-01-31, 모멘텀 인덱스 : 0.03588240046066504, flag : True, signal : buy S&P500
날짜 : 2008-02-29, 모멘텀 인덱스 : -0.00727349981897496, flag : False, signal : 
날짜 : 2008-03-31, 모멘텀 인덱스 : -0.04400731925250756, flag : False, signal : 
날짜 : 2008-04-30, 모멘텀 인덱스 : -0.09274341487643256, flag : False, signal : 
날짜 : 2008-05-30, 모멘텀 인덱스 : -0.0806840780935415, flag : False, signal : 
날짜 : 2008-06-30, 모멘텀 인덱스 : -0.05294098270428693, flag : False, signal : 
날짜 : 2008-07-31, 모멘텀 인덱스 : -0.10403985512817338, flag : False, signal : 
날짜 : 2008-08-29, 모멘텀 인덱스 : -0.12334059430356337, flag : False, signal : 
날짜 : 2008-09-30, 모멘텀 인덱스 : -0.14297231269240718, flag : False, signal : 
날짜 : 2008-10-31, 모멘텀 인덱스 : -0.2340724171379247, flag : False, signal : 
날짜 : 2008-11-28, 모멘텀 인덱스 : -0.3348296815075399, flag : False, signal : 
날짜 : 2008-12-31, 모멘텀 인덱스 : -0.37408169257794144, flag : False, signal : 
날짜 : 2009-01-30, 모멘텀 인덱스 : -0.3272762960666614, flag : False, signal : 
날짜 : 2009-02-27, 모멘텀 인덱스 : -0.3661361389216863,

In [7]:
# 전략 수익률
def returns(book, ticker):
    book['return'] = 1
    buy = 0.0
    signal = 'buy ' + ticker
    for i, x in enumerate(book.index):
        if book.loc[x, 'trade'] == signal and (i==0 or book.shift(1).loc[x, 'trade'] == ''):
            buy = book.loc[x, 'Close']
            print(f"진입일 : {x.strftime('%Y-%m-%d')}, 진입가격 : {buy}")
        elif buy != 0.0 and book.loc[x, 'trade'] == '' and book.shift(1).loc[x, 'trade'] == signal:
            sell = book.loc[x, 'Close']
            rtn = sell / buy
            print(f"청산일 : {x.strftime('%Y-%m-%d')}, 진입가격 : {buy}, 청산가격 : {sell}, return : {round(rtn, 4)}")
            buy = 0.0
        if book.shift(1).loc[x, 'trade'] == signal and i!=0:
            book.loc[x, 'return'] = book.loc[x, 'Close'] / book.shift(1).loc[x, 'Close']
    book['acc_ret'] = book['return'].cumprod()
    print(f"기간: {book.index[0].strftime('%Y/%m/%d')} ~ {book.index[-1].strftime('%Y/%m/%d')}")
    print(f"Accumulated return : {book.iloc[-1]['acc_ret']}")
    return round(book.iloc[-1]['acc_ret'], 4)

returns(book, ticker)
print(book.loc['2008':'2008-02-01'])

진입일 : 2008-01-02, 진입가격 : 107.151566
청산일 : 2008-02-01, 진입가격 : 107.151566, 청산가격 : 103.196136, return : 0.9631
진입일 : 2009-10-01, 진입가격 : 79.339211
청산일 : 2011-10-03, 진입가격 : 79.339211, 청산가격 : 88.092651, return : 1.1103
진입일 : 2011-11-01, 진입가격 : 97.764961
청산일 : 2012-01-03, 진입가격 : 97.764961, 청산가격 : 102.820366, return : 1.0517
진입일 : 2012-02-01, 진입가격 : 106.828362
청산일 : 2015-10-01, 진입가격 : 106.828362, 청산가격 : 167.135895, return : 1.5645
진입일 : 2015-11-02, 진입가격 : 183.020432
청산일 : 2016-02-01, 진입가격 : 183.020432, 청산가격 : 169.460709, return : 0.9259
진입일 : 2016-04-01, 진입가격 : 182.00705
청산일 : 2016-05-02, 진입가격 : 182.00705, 청산가격 : 182.930634, return : 1.0051
진입일 : 2016-06-01, 진입가격 : 184.953705
청산일 : 2019-01-02, 진입가격 : 184.953705, 청산가격 : 232.308777, return : 1.256
진입일 : 2019-02-01, 진입가격 : 250.768677
청산일 : 2020-04-01, 진입가격 : 250.768677, 청산가격 : 234.264236, return : 0.9342
진입일 : 2020-05-01, 진입가격 : 269.135101
청산일 : 2022-05-02, 진입가격 : 269.135101, 청산가격 : 406.066315, return : 1.5088
진입일 : 2023-04-03, 진입가격 : 409.429138


In [8]:
def get_evaluation(daily_return):
    """
    cagr, dd, mdd, vol, sharpe
    투자 성과 지표
    """
    # cumulativeReturn
    cumulativeReturn = daily_return.cumprod()
    # cagr
    cagr = cumulativeReturn.iloc[-1] ** (252/len(cumulativeReturn))
    # mdd
    dd = (cumulativeReturn.cummax() - cumulativeReturn) / cumulativeReturn.cummax() * 100
    mdd= dd.max()
    vol = np.std(daily_return-1) * np.sqrt(252)
    sharpe = np.mean(daily_return-1) * 252 / vol

    print(f"기간: {daily_return.index[0].strftime('%Y/%m/%d')} ~ {daily_return.index[-1].strftime('%Y/%m/%d')}")
    print(f"최종 수익률: {cumulativeReturn.iloc[-1]}\ncagr: {cagr}\nmdd: {mdd}\nvol: {vol}\nsharpe: {sharpe}")

    return cagr, dd, mdd, vol, sharpe

cagr, _, mdd, vol, sharp = get_evaluation(book.loc[:,'return'])

기간: 2008/01/02 ~ 2023/06/30
최종 수익률: 3.138518989953824
cagr: 1.0766829451705533
mdd: 33.71726379114351
vol: 0.14935511713345462
sharpe: 0.5698547171199796


### 4.3.2 듀얼 모멘텀 전략 구현을 위한 상대 모멘텀 전략

In [9]:
from datetime import timedelta
# s&p500,nasdaq,미국장기채,골드, 삼성전자,
stock_list = ['SPY', 'QQQ', 'TLT', 'GLD', '^KS11', 'CASH']
ticker_list = ['SPY', 'QQQ', 'TLT', 'GLD', 'KOSPI', 'CASH']
this_month = datetime(year=datetime.today().year, month=datetime.today().month, day=1).date()
base_df = yf.download(' '.join(stock_list[0:-1]), start='2007-01-01', end=this_month-timedelta(days=1))
base_df = base_df.fillna(method='ffill').dropna()
base_df.tail()

[*********************100%***********************]  5 of 5 completed


Adj Close                                                   \
                   GLD         QQQ         SPY         TLT        ^KS11   
Date                                                                      
2023-06-23  178.199997  362.540009  433.209991  103.330002  2570.100098   
2023-06-26  178.509995  357.679993  431.440002  103.440002  2582.199951   
2023-06-27  177.690002  363.829987  436.170013  103.169998  2581.389893   
2023-06-28  177.279999  364.540009  436.390015  103.610001  2564.189941   
2023-06-29  177.089996  363.809998  438.109985  101.739998  2550.020020   

                 Close                                                   ...  \
                   GLD         QQQ         SPY         TLT        ^KS11  ...   
Date                                                                     ...   
2023-06-23  178.199997  362.540009  433.209991  103.330002  2570.100098  ...   
2023-06-26  178.509995  357.679993  431.440002  103.440002  2582.199951  ...   
2023-06-27  177.690002  363.829987  436.170013  103.169998  2581.389893  ...   
2023-06-28  177.279999  364.540009  436.390015  103.610001  2564.189941  ...   
2023-06-29  177.089996  363.809998  438.109985  101.739998  2550.020020  ...   

                  Open                                                   \
                   GLD         QQQ         SPY         TLT        ^KS11   
Date                                                                      
2023-06-23  179.600006  362.209991  432.929993  103.830002  2599.040039   
2023-06-26  179.050003  362.000000  432.619995  103.620003  2568.649902   
2023-06-27  179.070007  359.250000  432.350006  103.589996  2581.020020   
2023-06-28  177.229996  361.980011  435.049988  103.410004  2590.520020   
2023-06-29  175.830002  364.239990  435.959991  102.169998  2564.020020   

               Volume                                                
                  GLD         QQQ         SPY         TLT     ^KS11  
Date                                                                 
2023-06-23  5416800.0  48873900.0  92074500.0  22896800.0  526500.0  
2023-06-26  3222100.0  52685500.0  72823600.0  12894900.0  479200.0  
2023-06-27  5036900.0  49428100.0  72813700.0  18105700.0  535400.0  
2023-06-28  5866700.0  46498000.0  75636000.0  23826800.0  484200.0  
2023-06-29  6862500.0  46151000.0  67882300.0  41091600.0  602500.0  

[5 rows x 30 columns]

In [10]:
stock_df = base_df['Adj Close'].copy()
stock_df['CASH'] = 1
stock_df.rename(columns=dict(zip(stock_list, ticker_list)), inplace=True)
print(stock_df.head())
print(stock_df.isna().sum(axis=0))

                  GLD        QQQ         SPY        TLT        KOSPI  CASH
Date                                                                      
2007-01-03  62.279999  37.685619  102.625740  55.019810  1409.349976     1
2007-01-04  61.650002  38.400288  102.843513  55.353447  1397.290039     1
2007-01-05  60.169998  38.217262  102.023209  55.112461  1385.760010     1
2007-01-08  60.480000  38.243385  102.495110  55.211330  1370.810059     1
2007-01-09  60.849998  38.435139  102.408012  55.211330  1374.339966     1
GLD      0
QQQ      0
SPY      0
TLT      0
KOSPI    0
CASH     0
dtype: int64


In [11]:
# 월말 데이터 추출
stock_df['STD_YM'] = stock_df.apply(lambda x: x.name.strftime('%Y-%m'), axis=1)
ym_keys = list(stock_df['STD_YM'].unique())
month_last_df = stock_df.drop_duplicates(['STD_YM'], keep="last").loc[:, ~stock_df.columns.isin(['STD_YM'])].copy()
print(month_last_df.head())

                  GLD        QQQ         SPY        TLT        KOSPI  CASH
Date                                                                      
2007-01-31  64.830002  38.408993  104.353455  54.086956  1360.229980     1
2007-02-28  66.480003  37.764050  102.306328  55.919907  1417.339966     1
2007-03-30  65.739998  37.962246  103.491966  54.969158  1452.550049     1
2007-04-30  67.089996  40.081448  108.076195  55.466591  1542.239990     1
2007-05-31  65.540001  41.345970  111.742157  54.183224  1700.910034     1


In [12]:
stock_df = stock_df.reset_index()
melt_stock_df = stock_df.melt(id_vars=['Date', 'STD_YM'], var_name='CODE', value_name='Close')
print(melt_stock_df.head(5))
print(melt_stock_df.tail(5))

        Date   STD_YM CODE      Close
0 2007-01-03  2007-01  GLD  62.279999
1 2007-01-04  2007-01  GLD  61.650002
2 2007-01-05  2007-01  GLD  60.169998
3 2007-01-08  2007-01  GLD  60.480000
4 2007-01-09  2007-01  GLD  60.849998
            Date   STD_YM  CODE  Close
25645 2023-06-23  2023-06  CASH    1.0
25646 2023-06-26  2023-06  CASH    1.0
25647 2023-06-27  2023-06  CASH    1.0
25648 2023-06-28  2023-06  CASH    1.0
25649 2023-06-29  2023-06  CASH    1.0


In [13]:
month_last_df = month_last_df.reset_index()
melt_return_month_df = month_last_df.melt(id_vars=['Date'], var_name='CODE', value_name='Close')
print(melt_return_month_df.head(5))
print(melt_return_month_df.tail(5))

        Date CODE      Close
0 2007-01-31  GLD  64.830002
1 2007-02-28  GLD  66.480003
2 2007-03-30  GLD  65.739998
3 2007-04-30  GLD  67.089996
4 2007-05-31  GLD  65.540001
           Date  CODE  Close
1183 2023-02-28  CASH    1.0
1184 2023-03-31  CASH    1.0
1185 2023-04-28  CASH    1.0
1186 2023-05-31  CASH    1.0
1187 2023-06-29  CASH    1.0


In [14]:
melt_return_month_df['1M_RET'] = melt_return_month_df['Close'] / melt_return_month_df.groupby('CODE')['Close'].shift()
melt_return_month_df.fillna(1, inplace=True)
print(melt_return_month_df.iloc[195:200])

          Date CODE       Close    1M_RET
195 2023-04-28  GLD  184.800003  1.008624
196 2023-05-31  GLD  182.320007  0.986580
197 2023-06-29  GLD  177.089996  0.971314
198 2007-01-31  QQQ   38.408993  1.000000
199 2007-02-28  QQQ   37.764050  0.983209


In [15]:
return_month_df = melt_return_month_df.pivot(index='Date', columns='CODE', values='1M_RET').copy()
rank_df = return_month_df.rank(axis=1, ascending=False, method='first')
# print(rank_df[rank_df['CASH'] < 4.0].head())
# print(return_month_df.loc[rank_df[rank_df['CASH'] < 4.0].index[0:2]].head())
buy_table = pd.DataFrame(np.where(rank_df < 4.0, 1, 0), index=rank_df.index, columns=rank_df.columns)[1:]
print(buy_table.head())

CODE        CASH  GLD  KOSPI  QQQ  SPY  TLT
Date                                       
2007-02-28     0    1      1    0    0    1
2007-03-30     0    0      1    1    1    0
2007-04-30     0    0      1    1    1    0
2007-05-31     0    0      1    1    1    0
2007-06-29     1    0      1    1    0    0


In [16]:
sig_dict = dict()
for date in buy_table.index:
    sig_dict[date] = list(buy_table.loc[date, buy_table.loc[date, :]>=1.0].index)
    # print(buy_table.loc[date, buy_table.loc[date, :]>=1.0].index)
    print(f"{date} : {sig_dict[date]}")

2007-02-28 00:00:00 : ['GLD', 'KOSPI', 'TLT']
2007-03-30 00:00:00 : ['KOSPI', 'QQQ', 'SPY']
2007-04-30 00:00:00 : ['KOSPI', 'QQQ', 'SPY']
2007-05-31 00:00:00 : ['KOSPI', 'QQQ', 'SPY']
2007-06-29 00:00:00 : ['CASH', 'KOSPI', 'QQQ']
2007-07-31 00:00:00 : ['GLD', 'KOSPI', 'TLT']
2007-08-31 00:00:00 : ['QQQ', 'SPY', 'TLT']
2007-09-28 00:00:00 : ['GLD', 'KOSPI', 'QQQ']
2007-10-31 00:00:00 : ['GLD', 'KOSPI', 'QQQ']
2007-11-30 00:00:00 : ['CASH', 'GLD', 'TLT']
2007-12-31 00:00:00 : ['CASH', 'GLD', 'QQQ']
2008-01-31 00:00:00 : ['CASH', 'GLD', 'TLT']
2008-02-29 00:00:00 : ['CASH', 'GLD', 'KOSPI']
2008-03-31 00:00:00 : ['CASH', 'QQQ', 'TLT']
2008-04-30 00:00:00 : ['KOSPI', 'QQQ', 'SPY']
2008-05-30 00:00:00 : ['KOSPI', 'QQQ', 'SPY']
2008-06-30 00:00:00 : ['CASH', 'GLD', 'TLT']
2008-07-31 00:00:00 : ['CASH', 'QQQ', 'TLT']
2008-08-29 00:00:00 : ['QQQ', 'SPY', 'TLT']
2008-09-30 00:00:00 : ['CASH', 'GLD', 'TLT']
2008-10-31 00:00:00 : ['CASH', 'QQQ', 'TLT']
2008-11-28 00:00:00 : ['CASH', 'GLD', 'TLT']

In [17]:
book = stock_df.set_index('Date').copy()
for c in ticker_list:
    book['p ' + c] = ''
    book['r ' + c] = ''
book.head()

,GLD,QQQ,SPY,TLT,KOSPI,CASH,STD_YM,p SPY,r SPY,p QQQ,r QQQ,p TLT,r TLT,p GLD,r GLD,p KOSPI,r KOSPI,p CASH,r CASH
Date,,,,,,,,,,,,,,,,,,,
2007-01-03,62.279999,37.685619,102.625740,55.019810,1409.349976,1,2007-01,,,,,,,,,,,,
2007-01-04,61.650002,38.400288,102.843513,55.353447,1397.290039,1,2007-01,,,,,,,,,,,,
2007-01-05,60.169998,38.217262,102.023209,55.112461,1385.760010,1,2007-01,,,,,,,,,,,,
2007-01-08,60.480000,38.243385,102.495110,55.211330,1370.810059,1,2007-01,,,,,,,,,,,,
2007-01-09,60.849998,38.435139,102.408012,55.211330,1374.339966,1,2007-01,,,,,,,,,,,,


In [18]:
for date,values in sig_dict.items():
    for stock in values:
        book.loc[date, 'p '+stock] = 'ready '+stock
book.loc['2007-02-26':'2007-03']

,GLD,QQQ,SPY,TLT,KOSPI,CASH,STD_YM,p SPY,r SPY,p QQQ,r QQQ,p TLT,r TLT,p GLD,r GLD,p KOSPI,r KOSPI,p CASH,r CASH
Date,,,,,,,,,,,,,,,,,,,
2007-02-26,68.099998,39.254387,105.384315,55.473328,1470.030029,1,2007-02,,,,,,,,,,,,
2007-02-27,65.410004,37.642036,101.268280,56.174267,1454.599976,1,2007-02,,,,,,,,,,,,
2007-02-28,66.480003,37.764050,102.306328,55.919907,1417.339966,1,2007-02,,,,,ready TLT,,ready GLD,,ready KOSPI,,,
2007-03-01,65.820000,37.589748,102.001427,55.884445,1417.339966,1,2007-03,,,,,,,,,,,,
2007-03-02,63.709999,37.023212,100.665756,56.164658,1417.339966,1,2007-03,,,,,,,,,,,,
2007-03-05,62.930000,36.735630,99.707474,56.258057,1376.150024,1,2007-03,,,,,,,,,,,,
2007-03-06,64.150002,37.345707,101.413383,56.089924,1402.930054,1,2007-03,,,,,,,,,,,,
2007-03-07,64.300003,37.197548,101.311798,56.307854,1410.949951,1,2007-03,,,,,,,,,,,,
2007-03-08,64.480003,37.450291,102.168365,56.258057,1423.890015,1,2007-03,,,,,,,,,,,,


In [19]:
def tradings(book, s_codes):
    std_ym = ''
    buy_phase = False
    for s in s_codes:
        for i in book.index:
            p_ticker = 'p ' + s
            if book.shift(1).loc[i, p_ticker] == 'ready '+s:
                std_ym = book.loc[i, 'STD_YM']
                buy_phase = True
            if book.loc[i, 'STD_YM'] != std_ym and book.shift(1).loc[i, p_ticker] != 'ready '+s:
                buy_phase = None
                std_ym = None
            if book.loc[i, p_ticker] != 'ready '+s and book.loc[i,'STD_YM'] == std_ym and buy_phase == True:
                book.loc[i, p_ticker] = 'buy '+ s

    return book
book = tradings(book, ticker_list)

In [20]:
book.loc['2007-06-27':'2007-09-02', list(map(lambda ticker: 'p ' + ticker, ticker_list))]

,p SPY,p QQQ,p TLT,p GLD,p KOSPI,p CASH
Date,,,,,,
2007-06-27,buy SPY,buy QQQ,,,buy KOSPI,
2007-06-28,buy SPY,buy QQQ,,,buy KOSPI,
2007-06-29,buy SPY,ready QQQ,,,ready KOSPI,ready CASH
2007-07-02,,buy QQQ,,,buy KOSPI,buy CASH
2007-07-03,,buy QQQ,,,buy KOSPI,buy CASH
2007-07-04,,buy QQQ,,,buy KOSPI,buy CASH
2007-07-05,,buy QQQ,,,buy KOSPI,buy CASH
2007-07-06,,buy QQQ,,,buy KOSPI,buy CASH
2007-07-09,,buy QQQ,,,buy KOSPI,buy CASH


In [21]:
def multi_returns(book, s_codes):
    buy_dict = {}
    sell_dict = {}
    for s in s_codes:
        book.loc[:, 'r ' + s] = 1

    for i in book.index:
        for s in s_codes:
            p_ticker = 'p ' + s
            r_ticker = 'r ' + s
            if book.loc[i, p_ticker] == 'buy '+s and book.shift(1).loc[i, p_ticker] == 'ready '+s and book.shift(2).loc[i, p_ticker] == '': # long
                buy_dict[s] = book.loc[i, s]
            elif book.loc[i, p_ticker] == '' and book.shift(1).loc[i, p_ticker] == 'buy '+s:
                sell_dict[s] = book.loc[i, s]
                rtn = sell_dict[s] / buy_dict[s]
                book.loc[i, r_ticker] = rtn
                print(f"개별 청산일 : {i}, 종목 코드 : {s}, 진입 가격 : {buy_dict[s]}, 청산 가격 : {sell_dict[s]}. return : {(rtn-1) * 100}%")
                sell_dict[s] = 0.0
                buy_dict[s] = 0.0
    acc_rtn = 1.0
    for i in book.index:
        rtn = 0.0
        count = 0
        for s in s_codes:
            p_ticker = 'p ' + s
            if book.loc[i, p_ticker] == '' and book.shift(1).loc[i, p_ticker] == 'buy '+s:
                count += 1
                rtn += book.loc[i, 'r '+s] - 1
        if (rtn != 0.0) and (count != 0.0):
            acc_rtn *= rtn/count + 1
            print(f"누적 청산일 : {i}, 청산 종목수 : {count}, 수익률 : {rtn/count}, 누적 수익률 : {acc_rtn}")

        book.loc[i, 'acc_rtn'] = acc_rtn
    print(f"누적 수익률 : {acc_rtn}")
multi_returns(book,ticker_list)

개별 청산일 : 2007-04-02 00:00:00, 종목 코드 : TLT, 진입 가격 : 55.88444519042969, 청산 가격 : 55.060394287109375. return : -1.4745621979645795%
개별 청산일 : 2007-04-02 00:00:00, 종목 코드 : GLD, 진입 가격 : 65.81999969482422, 청산 가격 : 65.8499984741211. return : 0.04557699701606133%
개별 청산일 : 2007-07-02 00:00:00, 종목 코드 : SPY, 진입 가격 : 103.60852813720703, 청산 가격 : 111.10388946533203. return : 7.234309243539316%
개별 청산일 : 2007-08-01 00:00:00, 종목 코드 : QQQ, 진입 가격 : 38.014583587646484, 청산 가격 : 41.80596160888672. return : 9.973482972656612%
개별 청산일 : 2007-08-01 00:00:00, 종목 코드 : CASH, 진입 가격 : 1, 청산 가격 : 1. return : 0.0%
개별 청산일 : 2007-09-03 00:00:00, 종목 코드 : GLD, 진입 가격 : 65.93000030517578, 청산 가격 : 66.5199966430664. return : 0.8948829594413121%
개별 청산일 : 2007-09-03 00:00:00, 종목 코드 : KOSPI, 진입 가격 : 1417.3399658203125, 청산 가격 : 1881.81005859375. return : 32.770549337089825%
개별 청산일 : 2007-10-01 00:00:00, 종목 코드 : SPY, 진입 가격 : 108.02963256835938, 청산 가격 : 113.47687530517578. return : 5.042359774175376%
개별 청산일 : 2007-10-01 00:00:00, 종목 

## 4.4 가치 투자 퀀트 전략

## 4.5 마치며